In [1]:
import compiler
from compiler.ast import *

def strip_parse(expr):
    return compiler.parse(expr).node.nodes[0].expr

/home/sudheera/dev/p4/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The compiler package is deprecated and removed in Python 3.x.
  """Entry point for launching an IPython kernel.


In [8]:
def sub_for_func(expr, func_name, func_vars, func_expr):
    """
    Return a string with the function func_name substituted for its exploded 
    form.
    
    func_name: The name of the function.
    func_vars: A sequence variables used by the function expression
    func_expr: The expression for the function.
    For example:
        If f(x, y, z) = sqrt(z)*x*y-z
        func_name = 'f'
        func_vars = ['x', 'y', 'z']
        func_expr = 'sqrt(z)*x*y-z'

    As a special case, functions that take a variable number of arguments can
    use '*' for func_vars.
    For example:
        sub_for_func('or_func(or_func(A,D),B,C)', 'or_func', '*', 'x or y')
        yields '(A or D) or B or C'
    """
    ast = strip_parse(expr)
    func_name_ast = strip_parse(func_name)
    if not isinstance(func_name_ast, Name):
        raise ValueError('Function name is not a simple name.')
    func_name = func_name_ast.name

    func_expr_ast = strip_parse(func_expr)
    # We can strip_parse  the '*', so we special case it here.
    if func_vars == '*':
        if not hasattr(func_expr_ast, 'nodes'):
            raise ValueError("Top-level function in %s does not appear to "
                             "accept variable number of arguments. (It has no "
                             "'nodes' attribute.)" % func_expr)

        func_var_names = '*'
    else:
        func_vars_ast = [strip_parse(var) for var in func_vars]
        for var_ast in func_vars_ast:
            if not isinstance(var_ast, Name):
                raise ValueError('Function variable is not a simple name.')
        func_var_names = [getattr(var_ast, 'name') for var_ast in func_vars_ast]

    ast = _sub_for_func_ast(ast, func_name, func_var_names, func_expr_ast)
#     simple = Simplify._simplify_ast(ast)
    return ast

def _sub_for_func_ast(ast, func_name, func_vars, func_expr_ast):
    """
    Return an ast with the function func_name substituted out.
    """
    if isinstance(ast, CallFunc) and ast2str(ast.node) == func_name\
       and func_vars == '*':
        working_ast = copy.deepcopy(func_expr_ast)
        new_args = [_sub_for_func_ast(arg_ast, func_name, func_vars, 
                                      func_expr_ast) for arg_ast in ast.args]
        # This subs out the arguments of the original function.
        working_ast.nodes = new_args
        return working_ast
    if isinstance(ast, CallFunc) and ast2str(ast.node) == func_name\
       and len(ast.args) == len(func_vars):
        # If our ast is the function we're looking for, we take the ast
        #  for the function expression, substitute for its arguments, and
        #  return
        working_ast = copy.deepcopy(func_expr_ast)
        mapping = {}
        for var_name, arg_ast in zip(func_vars, ast.args):
            subbed_arg_ast = _sub_for_func_ast(arg_ast, func_name, func_vars, 
                                               func_expr_ast)
            mapping[var_name] = subbed_arg_ast
        _sub_subtrees_for_vars(working_ast, mapping)
        return working_ast
    ast = AST.recurse_down_tree(ast, _sub_for_func_ast, 
                                (func_name, func_vars, func_expr_ast,))
    return ast

In [9]:
def test_sub_for_func():
    cases = [('f(x)', 'f', 'y', 'y+1','x+1')]

    for expr, func_name, func_vars, func_expr, answer in cases:
        print
        subbed = sub_for_func(expr, func_name, func_vars,
                                           func_expr)
        assert eval(answer) == eval(subbed)

In [10]:
test_sub_for_func()

NameError: global name 'ast2str' is not defined

In [15]:
x = "(2+4)/(3+8)"
def _collect_num_denom(ast, nums, denoms):
    """
    Append to nums and denoms, respectively, the nodes in the numerator and 
    denominator of an AST.
    """
    if not (isinstance(ast, Mul) or isinstance(ast, Div)):
        # If ast is not multiplication or division, just put it in nums.
        nums.append(ast)
        return

    if isinstance(ast.left, Div) or isinstance(ast.left, Mul):
        # If the left argument is a multiplication or division, descend into
        #  it, otherwise it is in the numerator.
        _collect_num_denom(ast.left, nums, denoms)
    else:
        nums.append(ast.left)

    if isinstance(ast.right, Div) or isinstance(ast.right, Mul):
        # If the left argument is a multiplication or division, descend into
        #  it, otherwise it is in the denominator.
        if isinstance(ast, Mul):
            _collect_num_denom(ast.right, nums, denoms)
        elif isinstance(ast, Div):
            # Note that when we descend into the denominator of a Div, we want 
            #  to swap our nums and denoms lists
            _collect_num_denom(ast.right, denoms, nums)
    else:
        if isinstance(ast, Mul):
            nums.append(ast.right)
        elif isinstance(ast, Div):
            denoms.append(ast.right)
print strip_parse(x)
_collect_num_denom(strip_parse(x), [], [])

Div((Add((Const(2), Const(4))), Add((Const(3), Const(8)))))


In [2]:
x=strip_parse('g(x-0+2, y**2 - 0**0, z*y + x/1)')
print(x)

CallFunc(Name('g'), [Add((Sub((Name('x'), Const(0))), Const(2))), Sub((Power((Name('y'), Const(2))), Power((Const(0), Const(0))))), Add((Mul((Name('z'), Name('y'))), Div((Name('x'), Const(1)))))], None, None)
